In [3]:
from __future__ import absolute_import, division, print_function

from os import environ, getcwd
from os.path import join

import keras
import numpy as np
import pandas as pd
import sklearn as skl
import tensorflow as tf
from keras.applications import NASNetMobile
from keras.layers import Dense, GlobalAveragePooling2D
from keras.metrics import binary_accuracy, binary_crossentropy
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

from mura import Mura

pd.set_option('display.max_rows', 20)
pd.set_option('precision', 4)
np.set_printoptions(precision=4)

environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Shut up tensorflow!
print("tf : {}".format(tf.__version__))
print("keras : {}".format(keras.__version__))
print("numpy : {}".format(np.__version__))
print("pandas : {}".format(pd.__version__))
print("sklearn : {}".format(skl.__version__))

# Hyper-parameters / Globals
BATCH_SIZE = 512  # tweak to your GPUs capacity
IMG_HEIGHT = 224  # ResNetInceptionv2 & Xception like 299, ResNet50/VGG/Inception 224, NASM 331
IMG_WIDTH = IMG_HEIGHT
CHANNELS = 3
DIMS = (IMG_HEIGHT, IMG_WIDTH, CHANNELS)  # blame theano
MODEL_TO_EVAL = '/Users/sidgupta/Downloads/NASNetMobile.hdf5'
DATA_DIR = '/Users/sidgupta/Documents/mura'
EVAL_CSV = 'valid.csv'
data_dir = DATA_DIR


EVAL_DIR = join(data_dir, 'valid')

# load up our csv with validation factors
data_dir = join(getcwd(), 'MURA-v1.1')
val_csv = join(data_dir, 'valid_labeled_studies.csv')
data_dir = join(getcwd(), DATA_DIR)
eval_csv = join(data_dir, EVAL_CSV)
df = pd.read_csv(eval_csv, names=['img', 'label'], header=None)
eval_imgs = df.img.values.tolist()
eval_labels = df.label.values.tolist()

eval_datagen = ImageDataGenerator(rescale=1. / 255)
eval_generator = eval_datagen.flow_from_directory(
    EVAL_DIR, class_mode='binary', shuffle=False, target_size=(IMG_HEIGHT, IMG_WIDTH), batch_size=BATCH_SIZE)
n_samples = eval_generator.samples
base_model = NASNetMobile(input_shape=DIMS, weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D(name='avg_pool')(x)  # comment for RESNET
x = Dense(1, activation='sigmoid', name='predictions')(x)
model = Model(inputs=base_model.input, outputs=x)
model.load_weights(MODEL_TO_EVAL)
model.compile(optimizer=Adam(lr=1e-3), loss=binary_crossentropy, metrics=['binary_accuracy'])
score, acc = model.evaluate_generator(eval_generator, n_samples / BATCH_SIZE)
print(model.metrics_names)
print('==> Metrics with eval')
print("loss :{:0.4f} \t Accuracy:{:0.4f}".format(score, acc))
y_pred = model.predict_generator(eval_generator, n_samples / BATCH_SIZE)
mura = Mura(eval_generator.filenames, y_true=eval_generator.classes, y_pred=y_pred)
print('==> Metrics with predict')
print(mura.metrics())
print(mura.metrics_by_encounter())
# print(mura.metrics_by_study_type())

tf : 1.10.0
keras : 2.2.4
numpy : 1.16.0
pandas : 0.23.4
sklearn : 0.20.2
Found 3197 images belonging to 2 classes.
['loss', 'binary_accuracy']
==> Metrics with eval
loss :0.4340 	 Accuracy:0.8164
==> Metrics with predict
per image metrics:
	accuracy : 0.82	f1 : 0.84	precision : 0.78	recall : 0.89	cohen_kappa : 0.63
per encounter metrics:
	accuracy : 0.83	f1 : 0.86	precision : 0.80	recall : 0.92	cohen_kappa : 0.63


In [1]:
from __future__ import absolute_import, division, print_function

from os import environ, getcwd
from os.path import join

import keras
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.applications import NASNetMobile
from keras.layers import Dense, GlobalAveragePooling2D
from keras.metrics import binary_accuracy, binary_crossentropy
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

from mura import Mura

#pd.set_option('display.max_rows', 20)
#pd.set_option('precision', 4)
#np.set_printoptions(precision=4)

#environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Shut up tensorflow!
#print("tf : {}".format(tf.__version__))
#print("keras : {}".format(keras.__version__))
#print("numpy : {}".format(np.__version__))
#print("pandas : {}".format(pd.__version__))

# Hyper-parameters / Globals
BATCH_SIZE = 512  # tweak to your GPUs capacity
IMG_HEIGHT = 224  # ResNetInceptionv2 & Xception like 299, ResNet50/VGG/Inception 224, NASM 331
IMG_WIDTH = IMG_HEIGHT
CHANNELS = 3
DIMS = (IMG_HEIGHT, IMG_WIDTH, CHANNELS)  # blame theano
MODEL_TO_EVAL = '/Users/sidgupta/Downloads/NASNetMobile.hdf5'
DATA_DIR = '/Users/sidgupta/Documents/mura'
EVAL_CSV = 'valid.csv'
data_dir = DATA_DIR


EVAL_DIR = join(data_dir, 'wrist')

# load up our csv with validation factors
data_dir = join(getcwd(), 'MURA-v1.1')
val_csv = join(data_dir, 'valid_labeled_studies.csv')
data_dir = join(getcwd(), DATA_DIR)
eval_csv = join(data_dir, EVAL_CSV)
df = pd.read_csv(eval_csv, names=['img', 'label'], header=None)
eval_imgs = df.img.values.tolist()
eval_labels = df.label.values.tolist()

eval_datagen = ImageDataGenerator(rescale=1. / 255)
eval_generator = eval_datagen.flow_from_directory(
    EVAL_DIR, class_mode='binary', shuffle=False, target_size=(IMG_HEIGHT, IMG_WIDTH), batch_size=BATCH_SIZE)
n_samples = eval_generator.samples
base_model = NASNetMobile(input_shape=DIMS, weights=None, include_top=False)
x = base_model.output
x = GlobalAveragePooling2D(name='avg_pool')(x)  # comment for RESNET
x = Dense(1, activation='sigmoid', name='predictions')(x)
model = Model(inputs=base_model.input, outputs=x)
model.load_weights(MODEL_TO_EVAL)
model.compile(optimizer=Adam(lr=1e-3), loss=binary_crossentropy, metrics=['binary_accuracy'])
score, acc = model.evaluate_generator(eval_generator, n_samples / BATCH_SIZE)
print(model.metrics_names)
print('==> Metrics with eval')
print("loss :{:0.4f} \t Accuracy:{:0.4f}".format(score, acc))
y_pred = model.predict_generator(eval_generator, n_samples / BATCH_SIZE)
mura = Mura(eval_generator.filenames, y_true=eval_generator.classes, y_pred=y_pred)
print('==> Metrics with predict')
print(mura.metrics())
print(mura.metrics_by_encounter())
# print(mura.metrics_by_study_type())

Using TensorFlow backend.


tf : 1.10.0
keras : 2.2.4
numpy : 1.16.0
pandas : 0.24.1
sklearn : 0.20.2
Found 9752 images belonging to 2 classes.
['loss', 'binary_accuracy']
==> Metrics with eval
loss :0.2752 	 Accuracy:0.8884
==> Metrics with predict
per image metrics:
	accuracy : 0.89	f1 : 0.91	precision : 0.87	recall : 0.96	cohen_kappa : 0.76
per encounter metrics:
	accuracy : 0.90	f1 : 0.92	precision : 0.88	recall : 0.97	cohen_kappa : 0.76


In [2]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
stem_conv1 (Conv2D)             (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn1 (BatchNormalization)   (None, 111, 111, 32) 128         stem_conv1[0][0]                 
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 111, 111, 32) 0           stem_bn1[0][0]                   
__________________________________________________________________________________________________
reduction_

In [6]:
from __future__ import absolute_import, division, print_function

from os import environ, getcwd
from os.path import join

import keras
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.applications import NASNetLarge
from keras.layers import Dense, GlobalAveragePooling2D
from keras.metrics import binary_accuracy, binary_crossentropy
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

from mura import Mura







In [7]:
# Hyper-parameters / Globals
BATCH_SIZE = 512  # tweak to your GPUs capacity
IMG_HEIGHT = 224  # ResNetInceptionv2 & Xception like 299, ResNet50/VGG/Inception 224, NASM 331
IMG_WIDTH = IMG_HEIGHT
CHANNELS = 3
DIMS = (IMG_HEIGHT, IMG_WIDTH, CHANNELS)  # blame theano
MODEL_TO_EVAL = '/Users/sidgupta/Downloads/NASNetMobile.hdf5'
DATA_DIR = '/Users/sidgupta/Documents/mura'
#EVAL_CSV = 'valid.csv'
data_dir = DATA_DIR


EVAL_DIR = join(data_dir, 'humerus')
PRED_DIR = join(data_dir, 'humerus_v')


In [8]:
eval_datagen = ImageDataGenerator(rescale=1. / 255)
eval_generator = eval_datagen.flow_from_directory(
    EVAL_DIR, class_mode='binary', shuffle=False, target_size=(IMG_HEIGHT, IMG_WIDTH), batch_size=BATCH_SIZE)
n_samples = eval_generator.samples
n_samples

Found 1272 images belonging to 2 classes.


1272

In [9]:
pred_datagen = ImageDataGenerator(rescale=1. / 255)
pred_generator = pred_datagen.flow_from_directory(
    PRED_DIR, class_mode='binary', shuffle=False, target_size=(IMG_HEIGHT, IMG_WIDTH), batch_size=BATCH_SIZE)
n_samples_v = pred_generator.samples

Found 288 images belonging to 2 classes.


In [11]:
base_model = NASNetLarge(input_shape=DIMS, weights=None, include_top=False)
x = base_model.output
x = GlobalAveragePooling2D(name='avg_pool')(x)  # comment for RESNET
x = Dense(1, activation='sigmoid', name='predictions')(x)
model = Model(inputs=base_model.input, outputs=x)
#model.load_weights(MODEL_TO_EVAL)
model.compile(optimizer=Adam(lr=1e-3), loss=binary_crossentropy, metrics=['binary_accuracy'])


In [12]:
score, acc = model.evaluate_generator(eval_generator, n_samples / BATCH_SIZE)
print(model.metrics_names)
print('==> Metrics with eval')
print("loss :{:0.4f} \t Accuracy:{:0.4f}".format(score, acc))


['loss', 'binary_accuracy']
==> Metrics with eval
loss :0.7418 	 Accuracy:0.4709


In [ ]:
y_pred = model.predict_generator(eval_generator, n_samples / BATCH_SIZE)
mura = Mura(eval_generator.filenames, y_true=eval_generator.classes, y_pred=y_pred)
print('==> Metrics with predict')
print(mura.metrics())
print(mura.metrics_by_encounter())
# print(mura.metrics_by_study_type())

==> Metrics with predict
per image metrics:
	accuracy : 0.47	f1 : 0.00	precision : 0.00	recall : 0.00	cohen_kappa : 0.00
per encounter metrics:
	accuracy : 0.46	f1 : 0.00	precision : 0.00	recall : 0.00	cohen_kappa : 0.00


/Users/sidgupta/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/sidgupta/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [41]:
y_pred = model.predict_generator(pred_generator, n_samples_v / BATCH_SIZE)
mura = Mura(pred_generator.filenames, y_true=pred_generator.classes, y_pred=y_pred)
print('==> Metrics with predict')
print(mura.metrics())
print(mura.metrics_by_encounter())
# print(mura.metrics_by_study_type())

==> Metrics with predict
per image metrics:
	accuracy : 0.86	f1 : 0.88	precision : 0.82	recall : 0.96	cohen_kappa : 0.71
per encounter metrics:
	accuracy : 0.88	f1 : 0.91	precision : 0.85	recall : 0.98	cohen_kappa : 0.71
